In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import os 
import sys
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [ ]:
#MAYBE INCLUDE day of the week, month, day of the year as input aswell

In [2]:
#TODO make supervised learning dataset from csv files
#TODO build dataloader
solar_data = np.genfromtxt('data\\training_data\\train_solar.csv', delimiter=',', skip_header=1)
wind_data = np.genfromtxt('data\\training_data\\train_wind.csv', delimiter=',',skip_header=1)

In [5]:
solar_data.shape, wind_data.shape

((17544, 11), (17544, 11))

In [10]:
def move_sliding_window(data, window_size, inputs_cols_indices, label_col_index):
    """
    data: numpy array including data
    window_size: size of window
    inputs_cols_indices: col indices to include
    """

    # (# instances created by movement, seq_len (timestamps), # features (input_len))
    inputs = np.zeros((len(data) - window_size, window_size, len(inputs_cols_indices)))
    labels = np.zeros(len(data) - window_size)

    for i in range(window_size, len(data)):
        inputs[i - window_size] = data[i - window_size : i, inputs_cols_indices]
        labels[i - window_size] = data[i, label_col_index]
    inputs = inputs.reshape(-1, window_size, len(inputs_cols_indices))
    labels = labels.reshape(-1, 1)
    print(inputs.shape, labels.shape)

    return inputs, labels

In [11]:
solar_X, solar_y = move_sliding_window(solar_data, 5, range(11), 0)
wind_X, wind_y = move_sliding_window(wind_data, 5, range(11), 1)

(17539, 5, 11) (17539, 1)
(17539, 5, 11) (17539, 1)


In [18]:
print(solar_X[0], '\n', solar_y[0])


[[-1.         -1.         -1.         -1.         -0.39492153  0.21077271
   0.          1.          1.          1.          1.        ]
 [-1.         -1.         -1.         -1.         -0.39191435  0.21077271
   1.          1.          1.          1.          1.        ]
 [-1.         -1.         -1.         -1.         -0.39321545  0.21077271
   2.          1.          1.          1.          1.        ]
 [-1.         -1.         -1.         -1.         -0.39499721  0.21077271
   3.          1.          1.          1.          1.        ]
 [-1.         -1.         -1.         -1.         -0.39659511  0.21077271
   4.          1.          1.          1.          1.        ]] 
 [-1.]


torch.Size([17537, 5, 6])
torch.Size([17537, 1])


In [15]:
#create dataset
class SolarDataset(Dataset):
    def __init__(self, inputs, output):
        self.inputs = inputs
        self.output = output

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.output[idx]

solar_dataset = SolarDataset(solar_X, solar_y)
wind_dataset = SolarDataset(wind_X, wind_y)
dataloader_solar = DataLoader(solar_dataset, batch_size=32, shuffle=False, drop_last=True) 
dataloader_wind = DataLoader(wind_dataset, batch_size=32, shuffle=False, drop_last=True)    


In [16]:
#save dataloader
torch.save(dataloader_solar, 'data/solar_dataloader.pth')
torch.save(dataloader_wind, 'data/wind_dataloader.pth')